### Climatology of temperature classification. 

Updated April 21, 2025. (I came back to do 3 classes based on standard deviation.) 

And again on April 23, 2025 to add in the additional areas. 

This file allows you to designate your desired area from the 2-degree gridded dataset I have on hand.

In [31]:
import datetime as dt
import numpy as np
import xarray as xr
import pickle

In [32]:
# temperature at 1000hPa over the full NH
infile = open("FullNH_1959_temps_2deg.p", 'rb') 
temp = pickle.load(infile)
infile.close()

In [33]:
print(temp.shape)

(62, 183, 46, 180)


In [34]:
##remove leap day
temp = np.delete(temp,[151],1)

In [35]:
##choose desired region
##for that europe area it is lon = (10,45 E) and lat = (60, 75 N)
###so ... to check the indexing I am quickly going to generate some arrays to check lol
lat = np.arange(90,-2,-2)
lon = np.arange(0,360,2)

print(len(lat))
print(len(lon))

46
180


European region of interest (60-75N, 10-45W). 

Nova Scotia (55-70N, 70-50W/290-310) 

SE US (32-40N, 90-105W/255-270). 

In [36]:
eur_lat = lat[7:16]
print(len(eur_lat))

eur_lon = lon[5:24]
print(len(eur_lon))

9
19


In [37]:
nova_lat = lat[10:18]
print(len(nova_lat))

nova_lon = lon[145:156]
print(len(nova_lon))

8
11


In [39]:
se_lat = lat[25:30]
print(len(se_lat))

se_lon = lon[127:136]
print(len(se_lon))

5
9


In [40]:
##designate area of northern europe
eur_t = temp[:,:,7:16,5:24]
print(eur_t.shape)

##designate area of nova scotia
nova_t = temp[:,:,10:18,145:156]
print(nova_t.shape)

##designate area of nova scotia
se_t = temp[:,:,25:30,127:136]
print(se_t.shape)

(62, 182, 9, 19)
(62, 182, 8, 11)
(62, 182, 5, 9)


In [42]:
##weighting
##create array... 
eur_t = xr.DataArray(data= eur_t, dims = ["year","day","lat","lon"],coords = dict(year = range(1959,2021,1),
                                                                                day = range(0,182,1),
                                                                                lat = eur_lat,
                                                                                lon = eur_lon))

nova_t = xr.DataArray(data= nova_t, dims = ["year","day","lat","lon"],coords = dict(year = range(1959,2021,1),
                                                                                day = range(0,182,1),
                                                                                lat = nova_lat,
                                                                                lon = nova_lon))

se_t = xr.DataArray(data= se_t, dims = ["year","day","lat","lon"],coords = dict(year = range(1959,2021,1),
                                                                                day = range(0,182,1),
                                                                                lat = se_lat,
                                                                                lon = se_lon))

In [45]:
##set definitions for weighting and the anomaly/median calculations...
def weights(target):
    weights=np.cos(np.deg2rad(target.lat))
    weight = target.weighted(weights).mean(dim="lon").mean(dim="lat") #average over the full area
    print(weight.shape)
    return weight;

def daily_anomaly(target):
    dailymean = np.nanmean(target,axis=1)
    anom = np.zeros_like(target)
    for t in np.arange(target.shape[1]):
         anom[:,t] = target[:,t] - dailymean
    print(anom.shape)
    return anom; 

def daily_median(target):
    dailymed = np.nanmedian(target,axis=1)
    med = np.zeros_like(target)
    for t in np.arange(target.shape[1]):
         med[:,t] = target[:,t] - dailymed
    print(med.shape)
    return med; 

In [46]:
#now calculate weights
e_t_weight = weights(eur_t)
n_t_weight = weights(nova_t)
s_t_weight = weights(se_t)

(62, 182)
(62, 182)
(62, 182)


In [47]:
#calculate daily anomalies from area-weighted values
anom_et = daily_anomaly(e_t_weight)
anom_nt = daily_anomaly(n_t_weight)
anom_st = daily_anomaly(s_t_weight)

#calculate daily medians from area-weighted daily anomalies
med_et = daily_median(anom_et)
med_nt = daily_median(anom_nt)
med_st = daily_median(anom_st)

(62, 182)
(62, 182)
(62, 182)
(62, 182)
(62, 182)
(62, 182)


### For this next subsection, change `te1` according to whichever area you wish to process. 

This should be done twice ... the first time for one category and the second for the other. 

In [93]:
##flatten
te1 = anom_et

te1 = np.reshape(te1, (62*182))
e_st = np.empty((62*182))

In [94]:
#check for NaNs
if np.any(np.isnan(te1)) or np.any(np.isinf(te1)):
    print("NaN or Inf values found in te1!")

In [95]:
#create empty arrays for neg and pos classification
pos = []
neg = []
neut = []

In [96]:
te1

array([7.68869382, 7.66011027, 8.45504321, ..., 4.88880549, 3.63451554,
       2.77087504])

In [97]:
std = np.std(te1)
-std/2

-2.193865399538695

In [98]:
#loop and classify anomalies
for i in range(0,62*182):
        if te1[i] > 0:
            e_st[i] = 1
            pos.append(1)
            
        elif te1[i] < 0:
            #print("negative")
            e_st[i] = 0
            neg.append(0)
            
        ##alternatine in an attempt to make arrays ... even? lol  
        elif te1[i] == 0:
            if len(pos) <= len(neg):
                e_st[i] = 1
                pos.append(1)
            else:
                e_st[i] = 0
                neg.append(0)

In [99]:
e_st

array([1., 1., 1., ..., 1., 1., 1.])

In [100]:
print(len(te1))
print(len(neg))
print(len(pos))
print(len(neut))

print(int(len(neg))+int(len(pos))+int(len(neut)))

11284
5659
5625
0
11284


In [101]:
pickle.dump(e_st, open("europetemps_anom_classed.p", 'wb'))